<center><img src="https://i.imgur.com/hkb7Bq7.png" width="500"></center>


### **Prof. José Manuel Magallanes, PhD**

* Professor, Departamento de Ciencias Sociales, Pontificia Universidad Católica del Perú, [jmagallanes@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe)

* Visiting Professor, Evans School of Public Policy and Governance / Senior Data Science Fellow, eScience Institute, University of Washington, [magajm@uw.edu](mailto:magajm@uw.edu)
_____

_____

<a id='home'></a>

# Introduction to Python

### Using Python for Pre Processing

In the session we will see the use of Python to:

1. Collect data as dataframes into Python

2. Preprocess a data frame:
    * [Fix column names](#fixcolnames)
    * [Subset data](#subset)    
    * [Look for non-standar missing values](#lookfornas)
    * [Missing values](#missingcellvalues)
    * [Format data types](#formatdtypes)


3. Merge both tables:
    * [Basic merge](#merging)
    * [Fuzzy merge](#fuzzmerging)


4. Prepare a file for further analysis
    * [Scaling](#scaling)
    * [Exporting](#exporting)



## 1. Collect data tables into Python

In [ ]:
# Location of data file
linkFile="https://github.com/eScienceWinterSchool/PythonSession/raw/master/data/HDR21-22_Statistical_Annex_HDI_Table.xlsx"

Reading in a table from a file using pandas, since it is an Excel file, I requires **openpyxl**:

In [ ]:
# available in my computer?
!pip show openpyxl

If not available, please go to Anaconda and install it. Once installed, or if available, continue:

In [ ]:
# choose the right function:
import pandas as pd

hdiFile=pd.read_excel(linkFile)

[Home](#home)
______

______

## 2.  Pre Processing

<a id='fixcolnames'></a>

### Fix column names

Notice that we do not have the right column names. So we need to save them before we go on:

In [ ]:
hdiFile.iloc[[3,4],:]

As you see, the column names are in different positions:

In [ ]:
# and
hdiFile.iloc[4,:2].to_list()

In [ ]:
hdiFile.iloc[3,2:].to_list()

In [ ]:
# save column names 
RealHeaders=hdiFile.iloc[4,:2].to_list()+hdiFile.iloc[3,2:].to_list()
# these are:
RealHeaders

In [ ]:
hdiFile.columns=RealHeaders
hdiFile.head()

Notice the repeated column names, and that we do not need the last three columns. Let's solve that:

In [ ]:
hdiFile.iloc[:,:-3]

In [ ]:
# then,
hdiFile=hdiFile.iloc[:,:-3]

We still have column names with missing values, let's get rid of those:

In [ ]:
BetterHeaders=hdiFile.columns.dropna().to_list()
#result
BetterHeaders

In [ ]:
#subsetting again
hdiFile=hdiFile.loc[:,BetterHeaders]
hdiFile.head(20)

Notice above that the columns:
* Have acronyms in parenthesis.
* Have spaces between words.

Let's see what can be done using _Regular Expressions_ (REGEX):

In [ ]:
# bye anything between parenthesis
hdiFile.columns.str.replace('\(.+\)',"", regex=True)

In [ ]:
# bye anything between parenthesis, bye leading-trailing spaces
hdiFile.columns.str.replace('\(.+\)',"", regex=True).str.strip()

In [ ]:
# bye anything between parenthesis, bye leading-trailing spaces, title case
hdiFile.columns.str.replace('\(.+\)',"", regex=True).\
                          str.strip().\
                          str.title()

Let's keep this last one for a while:

In [ ]:
#changing column names
hdiFile.columns=hdiFile.columns.str.replace('\(.+\)',"", regex=True).\
                          str.strip().\
                          str.title()
#so
hdiFile

Now, it is time to decide how the we want as the shorter column name:

* Same title without spaces:

In [ ]:
hdiFile.columns.str.replace(" ",'',regex=False)

* Some acronyms: Let's do this step by step.

In [ ]:
# each column names splitted:
[name.split() for name in hdiFile.columns[2::]]

In [ ]:
# first letter of each word
[[word[0] for word in name.split()] for name in hdiFile.columns[2::]]

In [ ]:
# final result
[''.join([word[0] for word in name.split()]) for name in hdiFile.columns[2::]]

Let's keep the last alternative:

In [ ]:
newNames=[''.join([word[0] for word in name.split()]) for name in hdiFile.columns[2::]]
hdiFile.columns=hdiFile.columns[:2].to_list()+newNames

Finally...

In [ ]:
hdiFile.head(20)

[Home](#home)
______

<a id='subset'></a>

### Subset data

After becoming familar with the data, we know we should keep countries with ranking value:

In [ ]:
hdiFile[~pd.isna(hdiFile['Hdi Rank'])]

In [ ]:
# then
hdiFile=hdiFile[~pd.isna(hdiFile['Hdi Rank'])]

#see
hdiFile

We still have a bad row data:

In [ ]:
hdiFile.drop(index=4)

We just drop it:

In [ ]:
hdiSubset=hdiFile.drop(index=4)
hdiSubset.reset_index(drop=True, inplace=True)
#see
hdiSubset

[Home](#home)
______

<a id='lookfornas'></a>

### Look for non-standar missing values

First check a cell that is full of non-word/non-digit characters:

Using **try**:

In [ ]:
for i in range(hdiSubset.shape[1]):
    try:
        print(hdiSubset.iloc[:,i][hdiSubset.iloc[:,i].str.fullmatch("\W+",na=False)])
    except:
        pass

We do not have weird symbols, but if we had:

In [ ]:
# replacing !

badSymbols=["..",'xx','tba']
hdiSubset.replace(to_replace=badSymbols,
               value=None,
               inplace=True)

#result
hdiSubset

[Home](#home)
______

<a id='missingcellvalues'></a>

### Missing values

Do the current cell values have issues?

* Keeping complete data

In [ ]:
# with all missing (after the first column)
hdiSubset[hdiSubset.iloc[:,1:].isna().all(axis=1)]

In [ ]:
# with at least one missing (after the first column)
hdiSubset[hdiSubset.iloc[:,1:].isna().any(axis=1)]

* Making sure columns of _text_ are clean:

In [ ]:
# get rid of leading and trailing spaces in text cells
hdiSubset.Country=hdiSubset.Country.str.strip()

* Checking  numeric columns

In [ ]:
hdiSubset.drop(columns=['Country'], axis=0).info()

Numbers have been recognised as **object** type. It might be due to having a non numeric value in one cell, or because it **had** a non-numeric value before.

In [ ]:
# can you apply math?
hdiSubset.drop(columns=['Country'], axis=0).max()

You just need to give format.

In [ ]:
hdiClean=hdiSubset.copy()

[Home](#home)
______

<a id='formatdtypes'></a>

### Formatting

From above, we just need to format the numeric columns:

* **Formatting into numeric type**:

In [ ]:
# keep all numeric columns

allCols=hdiClean.columns.to_list()
allCols.remove('Country')
allCols

In [ ]:
# as easy as:
hdiClean[allCols]=hdiClean[allCols].apply(pd.to_numeric)

In [ ]:
#recheck
hdiClean.info()

That was easy!

In [ ]:
hdiFormat=hdiClean.copy()

[Home](#home)
______


## 3. Integrating

Let me bring another table, this time from [wikipedia](https://en.wikipedia.org/wiki/The_Economist_Democracy_Index#Components). Make sure to have **html5lib** and **beautifulsoup4** installed before the next code:

In [ ]:
linkwiki='https://en.wikipedia.org/wiki/The_Economist_Democracy_Index'
sortableTables=pd.read_html(io=linkwiki,flavor='bs4',attrs={"class": "wikitable sortable"})

# we got
type(sortableTables), len(sortableTables)

In [ ]:
# let's see the third one
sortableTables[2]

In [ ]:
# this the one
demoTable=sortableTables[2].copy()

# check columns
demoTable.columns.to_list()

In [ ]:
demoTable.columns.str.contains('Δ')

In [ ]:
demoTable.columns[~demoTable.columns.str.contains('Δ')]

In [ ]:
# keep some:
someCols=demoTable.columns[~demoTable.columns.str.contains('Δ')]

#subset
demoSub=demoTable[someCols].copy()

#then
demoSub

In [ ]:
demoSub.info()

[Home](#home)
______

<a id='merging'></a>

### Basic merging



If we are confident we did a good cleaning and formatting, this step should be easy:

In [ ]:
# left_on= / right_on NOT NEEDED (only when column names differ)
hdiFormat.merge(demoSub,left_on='Country', right_on='Country')

Notice the amount of rowd **returned above**, and compare it with the amount of rows in each data frame:

In [ ]:
len(hdiFormat),len(demoSub)

If you do not want to check country names, you stop here.

[Home](#home)
______

<a id='fuzzmerging'></a>

### Fuzzy Merge


The smallest amount of rows between two tables, is the maximum amount you expect after the merge. Let's check the key values that were not matched:

In [ ]:
onlyHDI=set(hdiFormat.Country)-set(demoSub.Country)
onlyDEMO=set(demoSub.Country)-set(hdiFormat.Country)

In [ ]:
onlyHDI

In [ ]:
onlyDEMO

The previous objects (onlyDEMO, onlyHDI) inform the values not matched in the other data frames. 
If you want to recover some of these values, you may follow these steps (you may need to install **thefuzz**):

In [ ]:
from thefuzz import process as fz

# take a country from onlyDEMO
# and get the country that matches the most in OnlyHDI

[(demo,fz.extractOne(demo, onlyHDI)) for demo in sorted(onlyDEMO)]

You will not get the best outcome in this step, so you just need to keep the 'safe' matches:

In [ ]:
[(demo,fz.extractOne(demo, onlyHDI)) for demo in sorted(onlyDEMO) \
 if fz.extractOne(demo, onlyHDI)[1]>=90]

The next step is replace the cells values in one of the data frames.
For that, you need to create a **dictionary of changes**:

In [ ]:
# this dictionary is prepared for HDI data:
{demo:fz.extractOne(demo, onlyHDI)[0] for demo in sorted(onlyDEMO) \
 if fz.extractOne(demo, onlyHDI)[1]>=90}

In [ ]:
# NOW create the dict and make the changes
changesDEMO={demo:fz.extractOne(demo, onlyHDI)[0] for demo in sorted(onlyDEMO) \
             if fz.extractOne(demo, onlyHDI)[1]>=90}

# replace in democracy

demoSub.Country.replace(to_replace=changesDEMO,inplace=True)

In [ ]:
# did you get more rows?
hdiFormat.merge(demoSub)

If you redo this process, you may recover more rows. I will not do it here, but you are welcome to. 

In [ ]:
# hint: start with these two lines!
onlyHDI=set(hdiFormat.Country)-set(demoSub.Country)
onlyDEMO=set(demoSub.Country)-set(hdiFormat.Country)
#check
[(demo,fz.extractOne(demo, onlyHDI)) for demo in sorted(onlyDEMO)]

We can do some changes manually:

In [ ]:
changesDEMO={'Cape Verde':'Cabo Verde',
             'Czech Republic':'Czechia',
             'East Timor':'Timor-Leste',
             'Ivory Coast':"Côte d'Ivoire",
             'Laos':"Lao People's Democratic Republic", 
             'South Korea':'Korea (Republic of)',
            'Turkey':'Türkiye'}

# replace in democracy

demoSub.Country.replace(to_replace=changesDEMO,inplace=True)

That was the best that could be done:

In [ ]:
hdidem=hdiFormat.merge(demoSub)

The format should still be good:

In [ ]:
hdidem.info()

[Home](#home)
______


## Prepare file for further work

<a id='scaling'></a>

###  Scaling

It would be good to check the range of values of your numeric data. You can simply use **describe** (just requesting _min_ and _max_):

In [ ]:
hdidem.describe().loc[['min','max']].T #T for transposing

As you see different ranges, it would be good to request a **boxplot** (make sure to install **matplotlib** if not previously installed)

In [ ]:
import matplotlib.pyplot as plt

hdidem.plot(kind='box', rot=90,fontsize=5)
plt.semilogy();

Notice that our concern is the numeric data. In case of categorical it is unusual to worry about it, but some cases might need some thinking.

Let me get the column names of the numeric columns:

In [ ]:
import numpy as np

colsToScale = hdidem.select_dtypes([np.number]).columns

colsToScale

Time to produce new ranges (make sure you have previously install **scikit-learn**):

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_minmax = scaler.fit_transform(hdidem.loc[:,colsToScale].to_numpy())
df_scaled = pd.DataFrame(df_minmax, columns=colsToScale)

Let's explore the result:

In [ ]:

df_scaled.describe().loc[['min','max']].T 


In [ ]:
df_scaled.plot(kind='box', rot=90,fontsize=5);

Let's add a suffix to the column names:

In [ ]:
df_scaled.columns=df_scaled.columns+"_mM"

In [ ]:
# concat to the right (instead of bottom) with axis=1
pd.concat([hdidem,df_scaled],axis=1)

So this is our last version:

In [ ]:
hdidem_plus=pd.concat([hdidem,df_scaled],axis=1)

[Home](#home)
______


<a id='exporting'></a>

### Exporting

#### For future use in Python:

In [ ]:
hdidem_plus_plus.to_pickle("hdidem_plus.pkl")
# you will need: DF=pd.read_pickle("hdidem_plus.pkl")
# or:
# from urllib.request import urlopen
# DF=pd.read_pickle(urlopen("https://...../hdidem_plus.pkl"),compression=None)

#### For future  use in R:

In [ ]:
!pip show rpy2

In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(hdidem_plus,file="hdidem_plus.RDS")

#In R, you call it with: DF = readRDS("hdidem_plus.RDS")
#or, if read from cloud: DF = readRDS(url("https://...../hdidem_plus.RDS")